## Reply

### Question 1

此处`enumerate`为python中的特殊函数，主要用于python中的`for`循环。`enumerate`的输入为**可遍历对象**（在python中包括列表、元组、迭代器等），通过`enumerate`可以直接得到遍历过程中的**索引**和**值**。

In [1]:
lst = ['Hello', 'World', 'ZJU']

print('Without enumerate:', end = ' \t')
for element in lst:
    print(element, end = ', ')

print('\nWith enumerate:', end = ' \t')
for element in enumerate(lst):
    print(element, end = ', ')

Without enumerate: 	Hello, World, ZJU, 
With enumerate: 	(0, 'Hello'), (1, 'World'), (2, 'ZJU'), 

可以看到，在使用`enumerate`后，`element`的类型变为了元组，元组内第一个元素为索引，第二个为值。`main.py`的62行为python中的一种特殊机制，即将元组内元素直接赋值给左边变量：

In [3]:
a, b, c = lst
print(f'a: {a}')
print(f'b: {b}')
print(f'c: {c}')

a: Hello
b: World
c: ZJU


其中，62行使用`enumerate`是为了获得batch的索引`batch_idx`，在第69行中存在对`batch_idx`的操作，该段的目的是获取当前epoch内训练的进度以及相关信息。第80行没有是因为代码原作者认为此时不需要batch的索引，后续也未进行69行所对应的操作。

### Question 2


对于`dry_run`，参考`main.py`第109行。这是python中`argparse`库的用法。一般情况下，如果要运行一段python脚本，对应的命令应为：
```shell
python main.py
```
其中`main.py`为文件名称。通过`argparse`即可在该命令后加相关参数，并被程序读取。参考`main.py`第96-117行，若使用命令
```shell
python main.py --lr 0.001 --dry-run
```
此时，`args.lr = 0.001`，`args.dry_run = True`。`argparse`的使用大概包括四步：
- 导入argparse包：`import argparse`；
- 创建一个命令行解析器对象，即`ArgumentParser()`对象；
- 给解析器添加命令行参数，即调用`add_argument()`方法；
- 解析命令行的参数，即使用`parse_args()`解析添加的参数。
  
其余使用请自行百度 / Google。需要说明的是之前提到的`lr`和`dry-run`，二者对应的`add_argument()`参数设置存在区别：
- 对于`lr`，其需要传入具体数值；
- 对于`dry-run`，只需要在命令里添加，则`args.dry_run = True`，否则为`False`。

参考`main.py`第110行给出的`help`信息，若添加此参数，则从62行开始的`for`循环只会运行一次，即测试程序是否可以运行。结合73行的`break`同样可以得出此结论。

> 注：个人感觉他是想写`try-run`，大概是搞错了（乐），英语水平急需提高

### Question 3

参考原文档（[link](https://pytorch.org/docs/stable/generated/torch.nn.functional.nll_loss.html?highlight=nll_loss#torch.nn.functional.nll_loss)），`nll_loss`即为*负对数似然误差*（Negtive Log Likelihood Loss）。实际公式可参考[NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)（在此链接中，reduction的说明已经很清楚，可以看它写的公式。）

对于reduction，其本质是因为，对于pytorch，其自动求导的函数输出必须是标量，但是很多loss function其无法函数可能会先计算一个向量出来。如果`reduction = 'sum'`，则直接对此向量求和；若`reduction = 'mean'`，则求平均。


### Question 4

Reference: [`torchvision.transforms.Normalize`](https://pytorch.org/vision/stable/generated/torchvision.transforms.Normalize.html?highlight=transforms+normalize#torchvision.transforms.Normalize)

概括一下，整个`transforms`工具都是为了预处理图片（包括但不限于转换为`torch.Tensor`、旋转、裁剪、缩放、正则化）。`Normalize`的目的：正常来说一个RGB图像可以表示为`(3 * height * width)`的张量（height和width哪个在前我忘了，都一样），其中`3`表示的是R、G、B三个通道，每个通道对应矩阵的元素范围在0~255。如果直接把这个数值输入给神经网络，可能会导致神经网络不收敛，因此提前进行正则化处理。具体正则化计算公式以及参数含义参考上方链接。

至于这个数值，要么是这哥们随便搞得，要么是他玄学调参调的，不解释（

### Question 5

可以，这个函数的目的是为了调整学习过程中的学习率，也即优化步长，可以采用的其他函数参考此[链接](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

### Question 6

这个哥们估计真的是自己写着玩玩……这个代码格式一言难尽。整体来说：
- `main.py`：训练模型，并将模型保存到指定地方
- `fgsm.py`：加载在`main.py`中训练好的模型，对其进行FGSM攻击。
因此，在`main.py`中没有加载模型这一步，因为它的作用就是训练模型并保存，这种小模型也不需要从checkpoint加载后接着练。

`model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))`这种写法是强制将模型加载到CPU上。相对来说这种模型比较保险，不容易保存，一方面如果别人用的话可能他们自己的电脑没有NVIDIA显卡；另一方面如果没有指定`map_location`的话，会直接加载到GPU上，如果GPU显存不够就会报错。

至于`model.eval()`，严格来说，其含义是指此时该模型处于测试阶段，即评估模型在测试集上的性能表现。[`Dropout`](http://zh.d2l.ai/chapter_multilayer-perceptrons/dropout.html)作为一种防止过拟合的方法，其只在训练时发挥作用，而在测试时不发挥作用。当运行`model.eval()`后，模型中的Dropout不会发挥作用，以便对模型进行测试。
